### Transformers用于Translation的任务
Transformers库使用预训练的翻译模型（如BerMarianMET和MBart等）来完成翻译任务，这些任务基于自回归方式，逐词或者逐句生成目标语言的翻译。
#### 1. 编码器-解码器结构
* 编码器：接收输入的源语言文本并将其编码为隐含的表示（embedding），捕捉源语言文本的语义信息；
* 解码器：基于编码器输出的隐含表示，逐步生成目标语言的翻译文本。每生成一个新词时，解码器会使用已经生成的词作为输入，从而生成接下来的词。
#### 2. 自注意力机制
* 编码器和解码器中都使用了自注意力机制，允许模型在处理每个词时关注句子中其他词的含义，从而理解上下文关系。这对于翻译至关重要，因为单个词的含义往往取决于其周围的上下文。
#### 3. 交叉注意力
* 在生成目标语言时，解码器会使用交叉注意力机制关注编码器的输出。通过交叉注意力，解码器可以在生成每个词时动态地参考源语言的隐含表示。这使得生成的翻译更流畅和准确。
#### 4. 自回归生成
解码器使用自回归生成方式生成翻译：
* 初始时，解码器接收一个特殊的<BOS>（开始）标记，生成第一个词。
* 生成第一个词后，该词作为下一步的输入，生成第二个词。
* 这个过程不断重复，直到生成结束标记<EOS>，模型生成完整的翻译。
#### 5. Beam Search和其他解码策略
* Beam Search：保留多个可能的生成路径，每一步生成多个备选词，从中挑选最高评分的词组合。这种方法能提升生成文本的流畅性和准确度。
* 温度调节、Top-k和Top-p采样：通过控制词的选择范围增加多样性，使生成结果更自然。
#### 6. 预训练和微调
* 翻译模型通常通过大规模的平行语料（对齐的双语数据）进行预训练。例如，MarianMT 和 MBart 模型预训练时分别使用了多语言数据，使得它们可以适应多语言翻译需求。在具体任务中，模型也可以进行微调，以提高特定领域的翻译效果。

### 1. 导包

In [34]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
import pandas as pd
from transformers import DataCollatorForSeq2Seq

### 2. 加载模型和分词器

In [3]:
# 选择 T5 模型和分词器
model_name = "t5-small"  # 或者选择 "t5-base" 根据显存需求选择模型大小
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


### 3. 加载数据集

In [6]:
# 加载 WMT14 数据集，并选择英语到法语的翻译任务
dataset = load_dataset("wmt14", "fr-en", split="train[:1%]")  # 使用 1% 的数据来快速训练示例


In [9]:
dataset

Dataset({
    features: ['translation'],
    num_rows: 408367
})

In [11]:
dataset_df=pd.DataFrame(dataset['translation'])

In [12]:
dataset_df.head()

,en,fr
0,Resumption of the session,Reprise de la session
1,I declare resumed the session of the European ...,Je déclare reprise la session du Parlement eur...
2,"Although, as you will have seen, the dreaded '...","Comme vous avez pu le constater, le grand ""bog..."
3,You have requested a debate on this subject in...,Vous avez souhaité un débat à ce sujet dans le...
4,"In the meantime, I should like to observe a mi...","En attendant, je souhaiterais, comme un certai..."


### 4. 数据预处理

In [23]:
# 检查数据集结构
print(dataset)
print(dataset[0])  # 查看第一个样本的具体内容
# dataset只有一个translation列，并且该列包含嵌套的字典，其中en和fr分别对应源语言和目标语言的文本

Dataset({
    features: ['translation'],
    num_rows: 408367
})
{'translation': {'en': 'Resumption of the session', 'fr': 'Reprise de la session'}}


In [60]:
# 定义预处理函数
def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]  # 获取源语言文本
    targets = [ex["fr"] for ex in examples["translation"]]  # 获取目标语言文本
    # padding='max_length'表示将数据填充到最大长度
    model_inputs = tokenizer(inputs, max_length=128, padding='max_length', truncation=True)
    labels = tokenizer(targets, max_length=128, padding='max_length', truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 处理数据集
tokenized_dataset = dataset.map(preprocess_function, batched=True)


In [65]:
# 设置保留的列
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [66]:
tokenized_dataset

Dataset({
    features: ['translation', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 408367
})

In [67]:
tokenized_dataset['labels']

tensor([[ 7144,  7854,    20,  ...,     0,     0,     0],
        [ 1022, 30474, 21504,  ...,     0,     0,     0],
        [ 8192,   327,     3,  ...,     0,     0,     0],
        ...,
        [  622, 17361,     7,  ...,     0,     0,     0],
        [ 8786,  1292,   359,  ...,     0,     0,     0],
        [ 2228,  4550,  6577,  ...,     0,     0,     0]])

In [68]:
# 数据加载器
train_dataloader = DataLoader(tokenized_dataset, batch_size=4, shuffle=True)

### 5. 模型微调

In [57]:
# 选择优化器
optimizer = AdamW(model.parameters(), lr=5e-5)

# 微调模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

# 输出解读： T5ForConditionalGeneration模型的结构打印输出
# 1. shared：模型的共享嵌入层，大小为32128x512
# 2. encoder：
# （1）embed_tokens：用于将输入词嵌入到模型的高维空间中
# （2）block
# 3. final_layer_norm和dropout：在编码器的末端有一个层归一化层和一个dropout层
# 4. lm_head：线性层，将512维特诊转换回词汇表大小，用于生成输出词汇

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

### 6. 模型训练

没有GPU而且跑的速度太慢了，所以先不跑了！！！

In [69]:
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    for batch in tqdm(train_dataloader):
        inputs = {k: v for k, v in batch.items()}
        
        # 计算损失并优化
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    print(f"Loss at epoch {epoch + 1}: {loss.item()}")

Epoch 1


  1%|          | 1077/102092 [09:02<14:07:31,  1.99it/s]


KeyboardInterrupt: 